# Data preprocessing

In this notebook the data stored in `coinmarketcap.csv` is preprocessed.

In [1]:
# import modules
import os
import pandas as pd

## Constants

In [2]:
# directory of this projects root, jupyter must be started accordingly
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
# directory for the cache
CACHE_DIR = os.path.join(ROOT_DIR, "cache")
# resulting csv file holding **all** data
DATA_CSV = os.path.join(ROOT_DIR, "coinmarketcap.csv")

## Load and preprocess data

### Load `coinmarketcap.csv`

In [3]:
def loadCsv(path):
    """
    Load CSV specified by `path` as pandas dataframe.
    """
    return pd.read_csv(path)

### Require a currency to have at least `minSamples`

In [4]:
def filterMinSamples(df, minSamples):
    """
    Filter dataframe, remove currencies not having
    at least `minSamples`
    """
    grouped = df.groupby(["slug"]).size()
    sampleFilter = grouped[grouped >= minSamples]
    return df[df.slug.isin(sampleFilter.index)]

### Require a currency to have at least a volume of `minVolume` and a market capitalisation of `minMarketCap`

In [5]:
def filterMinVolumeAndMinMarketCap(df, minVolume, minMarketCap):
    """
    Filter dataframe, remove currencies not having
    `minVolume` and `minMarketCap`
    """
    names = df[(df.volume >= minVolume) &
            (df["marketcap"] >= minMarketCap)].slug.unique()
    return df[df.slug.isin(names)]

### Fill missing samples
Here we look for missing dates/samples for each currency and interpolate.

In [6]:
def fillMissingSamples(df):
    """
    Fill missing samples in dataframe.
    Each currency is checked if the time serie complete
    (if dates are missing).
    If the serie is not complete, the missing values are
    interpolated.
    """
    # count filled samples
    cnt = 0
    grouped = df.groupby(["slug"], sort=False)
    groups = []
    # for eac currency check the time serie
    for slug, group in grouped:
        # assure we have no duplicates
        assert(len(group.index) == len(group.index.unique()))
        name = group.name.unique()[0]
        # convert dates to datetime, may have missing dates
        index = pd.to_datetime(group["date"], format="%Y%m%d")
        # set index to datetime time serie
        group.set_index(index, inplace=True)
        # drop the date row
        group = group.drop("date", axis=1)
        # get the first and last date
        head = group.iloc[0].head(1).name
        tail = group.iloc[-1].head(1).name
        # create a datetime index holding continous dates
        # there are no missing dates in this index
        newIndex = pd.date_range(head, tail)
        # apply index to currency
        group = group.reindex(newIndex)
        # convert index of datetime to string representation
        date = group.index.strftime("%Y%m%d")
        # insert continous 'date' column
        group.insert(0, "date", date)
        # check if values are missing
        if group.isnull().values.any():
            group.slug = slug
            group.name = name
            # update counter
            cnt += len(group[group.isnull().any(axis=1)])
            # fill missing values
            group.interpolate(inplace=True)

        # here there should not be any missing values
        assert(not group.isnull().values.any())
        # drop the index, so we have the same format
        # as the original dataframe
        group = group.reset_index(drop=True)
        groups.append(group)

    print("Samples filled: {}".format(cnt))
    # concatenate all groups together to a new data frame
    return pd.concat(groups)

### Put all together into a nice function

In [7]:
def loadCoinMarketCap(
        minSamples=365, # require at least a year
        minVolume=1000*1000, # require a volume of at least 1 million
        minMarketCap=1000*1000, # require a market capitalisation of at least 1 million
        fillMissingDates=True, # by default, fill missing sample
        ):
    df = pd.read_csv(DATA_CSV)
    df = filterMinSamples(df, minSamples)
    df = filterMinVolumeAndMinMarketCap(df, minVolume, minMarketCap)

    # fill missing values
    if fillMissingDates:
        df = fillMissingSamples(df)

    # use date as index
    index = pd.to_datetime(df["date"], format="%Y%m%d")
    df.set_index(index, inplace=True)
    df = df.drop("date", axis=1)

    print("Loaded {} currencies, {} samples.".format(
        len(df.slug.unique()), len(df)))
    return df

## Run the code

In [8]:
df = loadCoinMarketCap()
df.head()

Samples filled: 3116
Loaded 239 currencies, 259544 samples.


,slug,name,open,high,low,close,volume,marketcap
date,,,,,,,,
2013-04-28,bitcoin,BTC,135.30,135.98,132.10,134.21,0.0,1.500520e+09
2013-04-29,bitcoin,BTC,134.44,147.49,134.00,144.54,0.0,1.491160e+09
2013-04-30,bitcoin,BTC,144.00,146.93,134.05,139.00,0.0,1.597780e+09
2013-05-01,bitcoin,BTC,139.00,139.89,107.72,116.99,0.0,1.542820e+09
2013-05-02,bitcoin,BTC,116.38,125.60,92.28,105.21,0.0,1.292190e+09
